# 디렉토리 설정 및 이미지데이터 전처리

In [1]:
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/CNN_upgrade/

/content/drive/MyDrive/CNN_upgrade


In [5]:
!pwd

/content/drive/MyDrive/CNN_upgrade


In [6]:
base_dir = './'
categories = ['datasets/pedestrian', 'datasets/sitter', 'datasets/taxier']
nb_classes = len(categories)

In [7]:
# 이미지 크기 지정
image_w = 256
image_h = 256
pixels = image_w*image_h*3

In [8]:
# 이미지데이터 읽어들이기
X = []
Y = []
for idx, category in enumerate(categories):
    # 레이블 지정
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    # 이미지
    image_dir = base_dir+category+'/*'
    files = glob.glob(image_dir)
    
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        X.append(data)
        Y.append(label)
    
X = np.array(X)
Y = np.array(Y)

# 학습 전용 데이터와 테스트 전용 데이터로 구분
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [9]:
# 데이터 정규화하기(0~1사이로)
X_train = X_train.astype("float") / 256
X_test  = X_test.astype("float")  / 256
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (674, 256, 256, 3)
X_test shape: (225, 256, 256, 3)


# 일반 CNN 모델 적용

In [10]:
#%tensorflow_version 1.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

# 모델 구조 정의 
model = Sequential()
model.add(Conv2D(32, (3, 3), (1, 1), input_shape=X_train.shape[1:], padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(128, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# 전결합층
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        1

In [11]:
# 모델 구축하기
from tensorflow.keras import metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.CategoricalAccuracy()])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=50)
mc = ModelCheckpoint(base_dir+'CNN_best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
histroy = model.fit(X_train, y_train, batch_size=16, epochs=50, callbacks=[es, mc], validation_split=0.15)

Epoch 1/50
36/36 [==============================] - 9s 52ms/step - loss: 1.9797 - categorical_accuracy: 0.2925 - val_loss: 1.0967 - val_categorical_accuracy: 0.3333
Epoch 2/50
36/36 [==============================] - 1s 36ms/step - loss: 1.1087 - categorical_accuracy: 0.3587 - val_loss: 1.0969 - val_categorical_accuracy: 0.3725
Epoch 3/50
36/36 [==============================] - 1s 33ms/step - loss: 1.0746 - categorical_accuracy: 0.5055 - val_loss: 1.0656 - val_categorical_accuracy: 0.5098
Epoch 4/50
36/36 [==============================] - 1s 32ms/step - loss: 1.0894 - categorical_accuracy: 0.3768 - val_loss: 1.0925 - val_categorical_accuracy: 0.3725
Epoch 5/50
36/36 [==============================] - 1s 33ms/step - loss: 1.0999 - categorical_accuracy: 0.3877 - val_loss: 1.0636 - val_categorical_accuracy: 0.5098
Epoch 6/50
36/36 [==============================] - 1s 33ms/step - loss: 1.0592 - categorical_accuracy: 0.4551 - val_loss: 1.0939 - val_categorical_accuracy: 0.3725
Epoch 7/50

In [13]:
# 모델 평가하기 
from tensorflow.keras.models import load_model
model = load_model(base_dir+'CNN_best_model.h5')
score = model.evaluate(X_test, y_test)
print('loss=', score[0])        # loss
print('accuracy=', score[1])    # acc

8/8 [==============================] - 1s 25ms/step - loss: 0.3613 - categorical_accuracy: 0.8933
loss= 0.3612544536590576
accuracy= 0.8933333158493042


In [14]:
from tensorflow.keras.models import load_model
model = load_model(base_dir+'CNN_best_model.h5')

for category in categories:
    for i in range(1,4):
        test_image = base_dir+category+'_test_'+str(i)+'.jpg'
        
        # 이미지 resize
        img = Image.open(test_image)
        img = img.convert("RGB")
        img = img.resize((256,256))
        data = np.asarray(img)
        X = np.array(data)
        X = X.astype("float") / 256
        X = X.reshape(-1, 256,256,3)
        
        # 예측
        pred = model.predict(X)
        print(pred)
        result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
        print(test_image, ':',categories[result[0]])
        print()

[[9.8279321e-01 1.7028950e-02 1.7781189e-04]]
./datasets/pedestrian_test_1.jpg : datasets/pedestrian

[[9.9991953e-01 2.8155808e-05 5.2302639e-05]]
./datasets/pedestrian_test_2.jpg : datasets/pedestrian

[[0.88421625 0.06906117 0.04672265]]
./datasets/pedestrian_test_3.jpg : datasets/pedestrian

[[9.954529e-01 7.781304e-05 4.469239e-03]]
./datasets/sitter_test_1.jpg : datasets/pedestrian

[[6.9750578e-04 9.9930060e-01 1.9316672e-06]]
./datasets/sitter_test_2.jpg : datasets/sitter

[[5.94272897e-05 9.99928236e-01 1.22570955e-05]]
./datasets/sitter_test_3.jpg : datasets/sitter

[[1.8235623e-05 2.6910951e-05 9.9995482e-01]]
./datasets/taxier_test_1.jpg : datasets/taxier

[[3.6759970e-06 2.9855844e-06 9.9999332e-01]]
./datasets/taxier_test_2.jpg : datasets/taxier

[[1.5344602e-07 2.0382144e-07 9.9999964e-01]]
./datasets/taxier_test_3.jpg : datasets/taxier

